First we need to load all the packages necessary for this notebook. In order to install the packages run pip install requirements.txt

In [1]:
import os
import pandas as pd
import warnings

Next we get the data using the environment variable RESEARCH_DATA_PATH which should direct to the data folder of this project

In [3]:
# Get the path to the data folder from the environment variable
DATA_PATH = os.environ.get("RESEARCH_DATA_PATH")

# Check if the DATA_PATH is not None
if DATA_PATH is not None:

    mai_daily_data = pd.read_csv(f'{DATA_PATH}/raw/mai_d_raw.csv')
    mai_monthly_data = pd.read_csv(f'{DATA_PATH}/raw/mai_m_raw.csv')
    mef_monthly_data = pd.read_csv(f'{DATA_PATH}/raw/mef_m_raw.csv')
    mkt_daily_data = pd.read_csv(f'{DATA_PATH}/raw/mkt_d_raw.csv')
    mkt_monthly_data = pd.read_csv(f'{DATA_PATH}/raw/mkt_m_raw.csv')

    print("Data loaded successfully!")
else:
    print("ERROR: The RESEARCH_DATA_PATH environment variable is not set.")


Data loaded successfully!


In [6]:
#Set date as index column
mai_daily_data.set_index('date', inplace=True)
mai_monthly_data.set_index('date', inplace=True)
mef_monthly_data.set_index('date', inplace=True)
mkt_daily_data.set_index('date', inplace=True)
mkt_monthly_data.set_index('date', inplace=True)

We notice that some values are missing (0)

In [7]:
#This function creates a table showing the total number of values, the number of missing values and the percentage of missing values

def missing(mai_dataframe):
    warnings.filterwarnings('ignore')
    missing_values = pd.DataFrame(columns=['Column', 'Total Count', 'Missing values', '% of missing values'])

    # Iterate through each column in the DataFrame
    for column in mai_dataframe.columns:
        total_count = mai_dataframe[column].count()  # Total number of non-NA/null entries
        zero_count = (mai_dataframe[column] == 0).sum()  # Count of zeros
        percentage_zeros = (zero_count / total_count) * 100  # Percentage of zeros

        # Append the results to the DataFrame
        missing_values = missing_values.append({
            'Column': column,
            'Total Count': total_count,
            'Missing values': zero_count,
            '% of missing values': percentage_zeros
        }, ignore_index=True)

    # Print the results
    print(missing_values)
    warnings.filterwarnings('default')


In [8]:
missing(mai_daily_data)

              Column Total Count Missing values  % of missing values
0   credit_rating_ni        8523           5844            68.567406
1             gdp_ni        8523           4957            58.160272
2       house_mkt_ni        8523           5593            65.622433
3       inflation_ni        8523           2454            28.792679
4        monetary_ni        8523           1780            20.884665
5             oil_ni        8523           2942            34.518362
6           unemp_ni        8523           2525            29.625719
7             usd_ni        8523           7271            85.310337
8   credit_rating_wi        8523           4600            53.971606
9             gdp_wi        8523           2205            25.871172
10      house_mkt_wi        8523           3882            45.547342
11      inflation_wi        8523            580             6.805116
12       monetary_wi        8523            747             8.764520
13            oil_wi        8523  

In order to get rid of some of the missing values, when an index value is missing from just one of the two sources (New York Times or Wall Street Journal) we use the non-missing value to fill in the missing one.

In [9]:
column_bases = ['credit_rating', 'gdp', 'house_mkt', 'inflation', 'monetary', 'oil', 'unemp', 'usd']

for base in column_bases:
    wi_col = f'{base}_wi'
    ni_col = f'{base}_ni'

    # Replace 0 in wi_col with value from ni_col if ni_col is not 0
    mask = (mai_daily_data[wi_col] == 0) & (mai_daily_data[ni_col] != 0)
    mai_daily_data.loc[mask, wi_col] = mai_daily_data.loc[mask, ni_col]

    # Replace 0 in ni_col with value from wi_col if wi_col is not 0
    mask = (mai_daily_data[ni_col] == 0) & (mai_daily_data[wi_col] != 0)
    mai_daily_data.loc[mask, ni_col] = mai_daily_data.loc[mask, wi_col]

In [10]:
missing(mai_daily_data)

              Column Total Count Missing values  % of missing values
0   credit_rating_ni        8523           3407            39.974187
1             gdp_ni        8523           1380            16.191482
2       house_mkt_ni        8523           3001            35.210607
3       inflation_ni        8523            223             2.616450
4        monetary_ni        8523            236             2.768978
5             oil_ni        8523            519             6.089405
6           unemp_ni        8523            604             7.086707
7             usd_ni        8523           2681            31.456060
8   credit_rating_wi        8523           3407            39.974187
9             gdp_wi        8523           1380            16.191482
10      house_mkt_wi        8523           3001            35.210607
11      inflation_wi        8523            223             2.616450
12       monetary_wi        8523            236             2.768978
13            oil_wi        8523  

Now we make the assumpion that for the values that are still missing, there was not any change in the corresponding index since the first non-missing value. So we use the last observed value to fill in the missing ones.

In [11]:
missing(mai_daily_data)

              Column Total Count Missing values  % of missing values
0   credit_rating_ni        8523           3407            39.974187
1             gdp_ni        8523           1380            16.191482
2       house_mkt_ni        8523           3001            35.210607
3       inflation_ni        8523            223             2.616450
4        monetary_ni        8523            236             2.768978
5             oil_ni        8523            519             6.089405
6           unemp_ni        8523            604             7.086707
7             usd_ni        8523           2681            31.456060
8   credit_rating_wi        8523           3407            39.974187
9             gdp_wi        8523           1380            16.191482
10      house_mkt_wi        8523           3001            35.210607
11      inflation_wi        8523            223             2.616450
12       monetary_wi        8523            236             2.768978
13            oil_wi        8523  

In [12]:
missing(mai_monthly_data)

              Column Total Count Missing values  % of missing values
0   credit_rating_ni         408            285            69.852941
1             gdp_ni         408            201            49.264706
2       house_mkt_ni         408            256            62.745098
3       inflation_ni         408            127            31.127451
4        monetary_ni         408             79            19.362745
5             oil_ni         408            161            39.460784
6           unemp_ni         408             97            23.774510
7             usd_ni         408            351            86.029412
8   credit_rating_wi         408            234            57.352941
9             gdp_wi         408             71            17.401961
10      house_mkt_wi         408            166            40.686275
11      inflation_wi         408             22             5.392157
12       monetary_wi         408             34             8.333333
13            oil_wi         408  

In [38]:
column_bases = ['credit_rating', 'gdp', 'house_mkt', 'inflation', 'monetary', 'oil', 'unemp', 'usd']

for base in column_bases:
    wi_col = f'{base}_wi'
    ni_col = f'{base}_ni'

    # Replace 0 in wi_col with value from ni_col if ni_col is not 0
    mask = (mai_monthly_data[wi_col] == 0) & (mai_monthly_data[ni_col] != 0)
    mai_monthly_data.loc[mask, wi_col] = mai_monthly_data.loc[mask, ni_col]

    # Replace 0 in ni_col with value from wi_col if wi_col is not 0
    mask = (mai_monthly_data[ni_col] == 0) & (mai_monthly_data[wi_col] != 0)
    mai_monthly_data.loc[mask, ni_col] = mai_monthly_data.loc[mask, wi_col]

In [39]:
missing(mai_monthly_data)

              Column Total Count Missing values  % of missing values
0   credit_rating_ni         408            180            44.117647
1             gdp_ni         408             46            11.274510
2       house_mkt_ni         408            116            28.431373
3       inflation_ni         408              9             2.205882
4        monetary_ni         408             10             2.450980
5             oil_ni         408             37             9.068627
6           unemp_ni         408             21             5.147059
7             usd_ni         408            111            27.205882
8   credit_rating_wi         408            180            44.117647
9             gdp_wi         408             46            11.274510
10      house_mkt_wi         408            116            28.431373
11      inflation_wi         408              9             2.205882
12       monetary_wi         408             10             2.450980
13            oil_wi         408  

In [40]:
# Replace 0 with NaN and then forward fill
mai_monthly_data.replace(0, pd.NA, inplace=True)
mai_monthly_data.fillna(method='ffill', inplace=True)

In [41]:
missing(mai_monthly_data)

              Column Total Count Missing values  % of missing values
0   credit_rating_ni         408              0                  0.0
1             gdp_ni         408              0                  0.0
2       house_mkt_ni         408              0                  0.0
3       inflation_ni         408              0                  0.0
4        monetary_ni         408              0                  0.0
5             oil_ni         408              0                  0.0
6           unemp_ni         408              0                  0.0
7             usd_ni         408              0                  0.0
8   credit_rating_wi         408              0                  0.0
9             gdp_wi         408              0                  0.0
10      house_mkt_wi         408              0                  0.0
11      inflation_wi         408              0                  0.0
12       monetary_wi         408              0                  0.0
13            oil_wi         408  